# Lab 4

In [1]:
import pandas as pd
import requests
from datetime import date
from io import StringIO

class ndawn_request:

    def __init__(self, startDate='YYYY-MM-DD', endDate='YYYY-MM-DD', ontology = None, location = None, save = False):

        self.start = startDate

        self.end = endDate

        # List of ontology terms, and their URL codes to build request URL
        self.ontology = {
            'Air Temperature': ['variable=hdt', 'variable=hdt9'],
            'Relative Humidity': ['variable=hdrh', 'variable=hdrh9'],
            'Soil Temperature': ['variable=hdbst', 'variable=hdtst'],
            'Wind Speed': ['variable=hdws', 'variable=hdmxws', 'variable=hdws10', 'variable=hdmxws10'],
            'Wind Direction': ['variable=hdwd', 'variable=hdsdwd', '&variable=hdwd10', 'variable=hdsdwd10'],
            'Solar Radiation': ['variable=hdsr'],
            'Rainfall': ['variable=hdr'],
            'Air Pressure': ['variable=hdbp'],
            'Dew Point': ['variable=hddp'],
            'Wind Chill': ['variable=hdwc']
        }
        # Concatenate the ontology keys into a list for exception printout later
        ontologiesErrorMessage = '\n'.join(list(self.ontology.keys()))

        # List of stations, and URL codes to build request URL
        self.stations = {
            'Ada': 78,
            'Becker': 118,
            'Campbell': 87,
            'Clarissa': 124,
            'Eldred': 2,
            'Fox': 93,
            'Greenbush': 70,
            'Hubbard': 119,
            'Humboldt': 4,
            'Kennedy': 82,
            'Little Falls': 120,
            'Mavie': 71,
            'Ottertail': 103,
            'Parkers Prairie': 116,
            'Perham': 114,
            'Perley': 3,
            'Pine Point': 115,
            'Rice': 121,
            'Roseau': 61,
            'Sabin': 60,
            'Staples': 122,
            'Stephen': 5,
            'Ulen': 91,
            'Wadena': 117,
            'Warren': 6,
            'Waukon': 92,
            'Westport': 123,
            'Williams': 95
        }
        # Concatenate station names into a list for exception printout later
        stationsErrorMessage = '\n'.join(list(self.stations.keys()))

        self.save = save

        # This checks the start and end dates supplied to make sure they are valid
        # Start by converting dates into iso format
        startDateCheck = date.fromisoformat(startDate)
        endDateCheck = date.fromisoformat(endDate)
        # If start date is after end date, raise exception
        if startDateCheck > endDateCheck:
            raise Exception('End date cannot be before start date')
        
        # Create empty list to hold URL codes for ontology terms
        self.activeMeasures = []
        # If user supplies ontology terms
        if ontology is not None:
            for item in ontology:
                # If user-supplied term is not in the dictionary, raise exception
                if item not in self.ontology.keys():
                    raise Exception('Ontology term [' + str(item) + '] not recognized. Available ontology terms include: ' + '\n' + ontologiesErrorMessage)
                # Otherwise, append URL codes for ontology terms into the list of measurements to be requested
                else:
                    for code in self.ontology[item]:
                        self.activeMeasures.append(code)  
        # If user does not supply ontology terms, add all URL codes in dictionary to the list of measurements to be requested    
        else:
            for key in self.ontology:
                for code in self.ontology[key]:
                    self.activeMeasures.append(code)

        # Create empty list to hold URL codes for stations
        self.activeStations = []
        # If user supplies station names
        if location is not None:
            for name in location:
                # If user-supplied name is not in the dictionary, raise exception
                if name not in self.stations.keys():
                    raise Exception('Station [' + str(name) + '] not recognized. Available stations include: ' + '\n' + stationsErrorMessage)
                # Otherwise, append URL codes for stations into the list of stations to be requested
                else:
                    self.activeStations.append('station=' + str(self.stations[name]))
        # If user does not supply station names, add all station URL codes in dictionary to the list of stations to be requested
        else:    
            for key in self.stations:
                self.activeStations.append('station=' + str(self.stations[key]))

    def get_data(self):
        
        # Construct API call for the request
        baseURL = 'https://ndawn.ndsu.nodak.edu/table.csv?'
        stations = '&'.join(self.activeStations)
        measurements = '&'.join(self.activeMeasures)
        options = '&ttype=hourly&quick_pick=&begin_date=' + self.start + '&end_date=' + self.end
        finalURL = str(baseURL + stations + '&' + measurements + options)
        
        # Request page
        page = requests.get(finalURL)
        # If status code not 200, raise exception
        if page.status_code != 200:
            raise Exception('URL request status not 200. Status code = ' + page.status_code)

        print('Request successful')

        # Convert csv data to string
        content = str(page.content)
        # Remove large, unnecessary header
        trimContent = content[content.find('Station'):len(content)]
        # Replace newline/return with string literal newline
        formatContent = trimContent.replace('\\r\\n', '\n')
        # Convert content to file object
        contentFile = StringIO(formatContent)

        # Read content into pandas dataframe. Second header row contains units
        ndawnData = pd.read_csv(contentFile, header = [0, 1])
        
        # Concatenate headers to include units
        # Assign column list to object
        columnHeaders = list(ndawnData.columns)
        # List of new headers
        newHeaderList = []
        # Iterate through column names
        for number in range(0, len(columnHeaders)):
            # If no unit, keep header unchanged, pass into new list
            if 'Unnamed' in columnHeaders[number][1]:
                newHeaderList.append(columnHeaders[number][0])
            # If unit exists, concatenate header and unit, pass into new list
            else:
                newHeader = columnHeaders[number][0] + ' (' + columnHeaders[number][1] + ') '
                newHeaderList.append(newHeader)
        # Assign new column names
        ndawnData.columns = newHeaderList

        # Create single column for datetime
        ndawnData['Date'] = pd.to_datetime(ndawnData[['Year', 'Month', 'Day']])
        
        # Save to csv if save option selected
        if self.save:
            ndawnData.to_csv('ndawnData.csv', index=False)

        return ndawnData
    
"""
# Example syntax:
exampleRequest = ndawn_request(startDate='2020-06-23', endDate='2020-06-28', ontology=['Air Pressure', 'Relative Humidity', 'Soil Temperature', 'Wind Direction', 'Wind Speed'], location=['Mavie', 'Ottertail', 'Perham', 'Perley'])
ndawnDF = exampleRequest.get_data()
"""

"\n# Example syntax:\nexampleRequest = ndawn_request(startDate='2020-06-23', endDate='2020-06-28', ontology=['Air Pressure', 'Relative Humidity', 'Soil Temperature', 'Wind Direction', 'Wind Speed'], location=['Mavie', 'Ottertail', 'Perham', 'Perley'])\nndawnDF = exampleRequest.get_data()\n"

In [2]:
# Creating the request and entering the parameters I am using for my work
r = ndawn_request(startDate = '2021-11-05', endDate = '2021-12-04', ontology = ['Air Temperature'], location =['Ada','Becker', 'Campbell', 'Clarissa','Eldred','Fox','Greenbush','Hubbard','Humboldt','Kennedy','Little Falls','Mavie','Ottertail','Parkers Prairie','Perham','Perley','Pine Point','Rice','Roseau','Sabin','Staples','Stephen','Ulen','Wadena','Warren','Waukon','Westport','Williams'])

In [3]:
# Calling the request
ndawnDF = r.get_data()

Request successful


In [6]:
# Viewing the first 5 rows of the dataframe to see if the data was successfully downloaded
ndawnDF.head()

,Station Name,Latitude (deg),Longitude (deg),Elevation (ft),Year,Month,Day,Hour (CST),Avg Air Temp (Degrees F),Avg Air Temp Flag,Avg Air Temp at 9 m (Degrees F),Avg Air Temp at 9 m Flag,Date
0,Ada,47.3211,-96.5139,910.0,2021.0,11.0,5.0,100.0,38.059,NaN,NaN,NaN,2021-11-05
1,Ada,47.3211,-96.5139,910.0,2021.0,11.0,5.0,200.0,36.837,NaN,NaN,NaN,2021-11-05
2,Ada,47.3211,-96.5139,910.0,2021.0,11.0,5.0,300.0,37.479,NaN,NaN,NaN,2021-11-05
3,Ada,47.3211,-96.5139,910.0,2021.0,11.0,5.0,400.0,35.807,NaN,NaN,NaN,2021-11-05
4,Ada,47.3211,-96.5139,910.0,2021.0,11.0,5.0,500.0,35.910,NaN,NaN,NaN,2021-11-05


In [8]:
#Creating a csv table of the data
temp_table = ndawnDF.to_csv('ndawnData.csv', index=False)

In [ ]:
# Adding the CSV to the workspace
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\lab4.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\lab4.gdb"):
    arcpy.conversion.TableToTable(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\lab4.gdb", "ndawn_Data", '', r'Station Name "Station Name" true true false 8000 Text 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Station Name,0,8000;Latitude (deg) "Latitude (deg)" true true false 8 Double 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Latitude (deg),-1,-1;Longitude (deg) "Longitude (deg)" true true false 8 Double 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Longitude (deg),-1,-1;Elevation (ft) "Elevation (ft)" true true false 8 Double 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Elevation (ft),-1,-1;Year "Year" true true false 8 Double 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Year,-1,-1;Month "Month" true true false 8 Double 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Month,-1,-1;Day "Day" true true false 8 Double 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Day,-1,-1;Hour (CST) "Hour (CST)" true true false 8 Double 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Hour (CST),-1,-1;Avg Air Temp (Degrees F) "Avg Air Temp (Degrees F)" true true false 8 Double 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Avg Air Temp (Degrees F),-1,-1;Avg Air Temp Flag "Avg Air Temp Flag" true true false 8000 Text 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Avg Air Temp Flag,0,8000;Avg Air Temp at 9 m (Degrees F) "Avg Air Temp at 9 m (Degrees F)" true true false 8000 Text 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Avg Air Temp at 9 m (Degrees F),0,8000;Avg Air Temp at 9 m Flag "Avg Air Temp at 9 m Flag" true true false 8000 Text 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Avg Air Temp at 9 m Flag,0,8000;Date "Date" true true false 8 Date 0 0,First,#,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\ndawnData.csv,Date,-1,-1', '')

In [ ]:
# Creating Points for each station at each date
arcpy.management.XYTableToPoint("ndawn_Data", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\lab4.gdb\ndawn_Data_XYTableToPoint1", "Longitude__deg_", "Latitude__deg_", "Avg_Air_Temp__Degrees_F_", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],VERTCS["WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')

## IDW

In [9]:
# Running IDW
arcpy.ddd.Idw("stations_avg_temp", "MEAN_Avg_Air_Temp__Degrees_F_", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\lab4.gdb\Idw_ndawn_Da1", 0.0132, 2, "VARIABLE 4", None)

<Result 'C:\\Users\\ecava\\OneDrive\\Documents\\ArcGIS\\Projects\\lab4\\lab4.gdb\\Idw_ndawn_Da1'>

## Kriging

In [10]:
#Ordinary using Spherical
arcpy.ddd.Kriging("ndawn_Data_XYTableToPoint1", "Avg_Air_Temp__Degrees_F_", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\lab4.gdb\Kriging", "Spherical 0.013200 # # #", 0.0132, "VARIABLE 4", None)

id,value
0,C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\lab4.gdb\Kriging_ndaw1
1,


## GPI

In [ ]:
# Performing GPI method
arcpy.ga.GlobalPolynomialInterpolation("stations_avg_temp", "MEAN_Avg_Air_Temp__Degrees_F_", None, r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\lab4\lab4.gdb\GPI_ndawn", 0.0132, 1, None)